In [15]:
%matplotlib inline

import backtrader as bt


class MultiIndicatorStrategy(bt.Strategy):
    """
    This strategy uses pre-computed technical indicators from our custom data feed:
      - 8, 13, 21 EMA for trend signals,
      - MACD (with its Signal Line) for momentum confirmation,
      - RSI for momentum/overbought filtering.
    
    Entry (Buy) Condition:
      - EMA_8 is above both EMA_13 and EMA_21,
      - MACD is positive,
      - RSI is below 70.
      
    Exit (Sell) Condition:
      - EMA_8 falls below both EMA_13 and EMA_21.
    """
    
    def __init__(self):
        # Use the pre-calculated indicators from the custom feed.
        self.ema8 = self.data.EMA_8
        self.ema13 = self.data.EMA_13
        self.ema21 = self.data.EMA_21
        
        self.macd = self.data.MACD
        self.signal_line = self.data.Signal_Line  # Not used in the entry, but available
        
        self.rsi = self.data.RSI_14

    def next(self):
        # Check if we are in the market
        if not self.position:
            # Long Entry:
            # - 8 EMA is above both 13 and 21 EMAs,
            # - MACD is positive,
            # - RSI is below 70.
            if (self.ema8[0] > self.ema13[0] and self.ema8[0] > self.ema21[0]
                and self.macd[0] > 0 and self.rsi[0] < 70
                ):
                self.buy()
                self.log("BUY CREATE, Price: {:.2f}".format(self.data.close[0]))
        else:
            # Exit (sell) when 8 EMA falls below both 13 and 21 EMAs.
            if self.ema8[0] < self.ema13[0] and self.ema8[0] < self.ema21[0]:
                self.close()
                self.log("SELL CREATE, Price: {:.2f}".format(self.data.close[0]))

    def log(self, txt, dt=None):
        """ Logging function for this strategy """
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()} - {txt}")


In [16]:
import pandas as pd

# Create a custom data feed class that maps your DataFrame columns.
class CustomPandasData(bt.feeds.PandasData):
    # Define additional lines that your DataFrame contains.
    lines = (
        'return', 'log_return', 'hourly_volatility',
        'EMA_8', 'EMA_13', 'EMA_21', 'EMA_signal',
        'EMA_short', 'EMA_long',
        'MACD', 'Signal_Line', 'MACD_Hist',
        'SMA20', 'STD20', 'Upper_BB', 'Lower_BB',
        'RSI_14'
    )
    # Map the DataFrame columns to Backtrader parameters.
    params = (
        ('datetime', None),        # use DataFrame index if datetime
        ('open', 'OPEN'),
        ('high', 'HIGH'),
        ('low', 'LOW'),
        ('close', 'CLOSE'),
        ('volume', 'VOLUME'),
        ('openinterest', -1),      # not used; set to -1

        # Map the additional columns
        ('return', 'return'),
        ('log_return', 'log_return'),
        ('hourly_volatility', 'hourly_volatility'),
        ('EMA_8', 'EMA_8'),
        ('EMA_13', 'EMA_13'),
        ('EMA_21', 'EMA_21'),
        ('EMA_signal', 'EMA_signal'),
        ('EMA_short', 'EMA_short'),
        ('EMA_long', 'EMA_long'),
        ('MACD', 'MACD'),
        ('Signal_Line', 'Signal_Line'),
        ('MACD_Hist', 'MACD_Hist'),
        ('SMA20', 'SMA20'),
        ('STD20', 'STD20'),
        ('Upper_BB', 'Upper_BB'),
        ('Lower_BB', 'Lower_BB'),
        ('RSI_14', 'RSI_14'),
    )

# Example: Load your CSV file into a pandas DataFrame.
# (Assuming your CSV has a datetime column as the index or one of the columns.)
df = pd.read_csv('../data/processed/btc_usdt_hourly_processed.csv', index_col='datetime', parse_dates=True)

# Create an instance of the custom data feed using your DataFrame.
data_feed = CustomPandasData(dataname=df)

# Now set up Cerebro and add the data feed.
cerebro = bt.Cerebro()
cerebro.addstrategy(MultiIndicatorStrategy)
cerebro.adddata(data_feed)

# (Optional) Set initial cash, commission, etc.
cerebro.broker.setcash(10000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
cerebro.broker.setcommission(commission=0.001)

print("Data feed successfully added to Cerebro!")

print("Starting Portfolio Value: {:.2f}".format(cerebro.broker.getvalue()))

Data feed successfully added to Cerebro!
Starting Portfolio Value: 10000.00


In [ ]:

cerebro.run()
print("Final Portfolio Value: {:.2f}".format(cerebro.broker.getvalue()))

#havent figured out plotting yet
cerebro.plot(style='candlestick', path='../plots/multi_indicator_strategy.png')

2024-02-24 - BUY CREATE, Price: 51139.30
2024-02-26 - SELL CREATE, Price: 51432.92
2024-02-27 - BUY CREATE, Price: 56745.66
2024-02-29 - SELL CREATE, Price: 61374.94
2024-03-01 - BUY CREATE, Price: 62129.15
2024-03-02 - SELL CREATE, Price: 61939.64
2024-03-03 - BUY CREATE, Price: 62393.86
2024-03-05 - SELL CREATE, Price: 65575.20
2024-03-06 - BUY CREATE, Price: 65929.50
2024-03-07 - SELL CREATE, Price: 65843.57
2024-03-07 - BUY CREATE, Price: 66665.41
2024-03-10 - SELL CREATE, Price: 68418.59
2024-03-11 - BUY CREATE, Price: 71019.99
2024-03-12 - SELL CREATE, Price: 70507.37
2024-03-13 - BUY CREATE, Price: 71980.00
2024-03-14 - SELL CREATE, Price: 71884.81
2024-03-17 - BUY CREATE, Price: 68021.79
2024-03-18 - SELL CREATE, Price: 66996.69
2024-03-21 - BUY CREATE, Price: 66654.45
2024-03-21 - SELL CREATE, Price: 65133.67
2024-03-22 - BUY CREATE, Price: 66448.08
2024-03-22 - SELL CREATE, Price: 65481.99
2024-03-23 - BUY CREATE, Price: 64596.06
2024-03-24 - SELL CREATE, Price: 64470.00
2024

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]